In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pyttsx3
from gtts import gTTS
import os
from pydub import AudioSegment

class TextToSpeechApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Text-to-Speech Application")
        
        # Text Input
        self.text_label = tk.Label(root, text="Enter Text:")
        self.text_label.pack()
        self.text_input = tk.Text(root, height=10, width=50)
        self.text_input.pack()
        
        # Language Selection
        self.language_label = tk.Label(root, text="Select Language:")
        self.language_label.pack()
        self.language_var = tk.StringVar(value="en")
        self.language_menu = ttk.Combobox(root, textvariable=self.language_var)
        self.language_menu['values'] = ('en', 'es', 'fr', 'de', 'it', 'ta') 
        self.language_menu.bind("<<ComboboxSelected>>", self.update_voice_options)
        self.language_menu.pack()
        
        # Voice Selection
        self.voice_label = tk.Label(root, text="Select Voice:")
        self.voice_label.pack()
        self.voice_var = tk.StringVar(value="Female")
        self.voice_menu = ttk.Combobox(root, textvariable=self.voice_var)
        self.voice_menu.pack()
        
        # Speech Rate Adjustment
        self.rate_label = tk.Label(root, text="Adjust Rate:")
        self.rate_label.pack()
        self.rate_scale = tk.Scale(root, from_=100, to=300, orient='horizontal')
        self.rate_scale.pack()
        
        # Volume Adjustment
        self.volume_label = tk.Label(root, text="Adjust Volume:")
        self.volume_label.pack()
        self.volume_scale = tk.Scale(root, from_=0, to=1, resolution=0.1, orient='horizontal')
        self.volume_scale.pack()
        
        # Playback Controls
        self.play_button = tk.Button(root, text="Play", command=self.play)
        self.play_button.pack()
        
        self.pause_button = tk.Button(root, text="Pause", command=self.pause)
        self.pause_button.pack()
        
        self.stop_button = tk.Button(root, text="Stop", command=self.stop)
        self.stop_button.pack()
        
        self.save_button = tk.Button(root, text="Save", command=self.save)
        self.save_button.pack()

        self.engine = pyttsx3.init()
        self.audio_file = "temp_audio.mp3"
        # gTTS uses the same voice for Male and Female
        self.voices = {
            'en': ['Male', 'Female'],
            'es': ['Male', 'Female'],
            'fr': ['Male', 'Female'],
            'de': ['Male', 'Female'],
            'it': ['Male', 'Female'],
            'ta': ['Male', 'Female']  
        }
        self.update_voice_options()

    def update_voice_options(self, event=None):
        language = self.language_var.get()
        self.voice_menu['values'] = self.voices.get(language)
        #self.voice_var.set('Male')
        
    def play(self):
        text = self.text_input.get("1.0", tk.END).strip()
        language = self.language_var.get()
        voice = self.voice_var.get()
        rate = self.rate_scale.get()
        volume = self.volume_scale.get()
        
        if not text:
            messagebox.showwarning("Input Error", "Please enter text to convert to speech.")
            return
        
        if language == "en":
            self.engine.setProperty('rate', rate)
            self.engine.setProperty('volume', volume)
            voices = self.engine.getProperty('voices')
            self.engine.setProperty('voice', voices[0].id if voice == "Male" else voices[1].id)
            self.engine.say(text)
            self.engine.runAndWait()
        else:
            try:
                tts = gTTS(text=text, lang=language, slow=False)
                tts.save(self.audio_file)
                os.system(f"start {self.audio_file}")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to generate speech: {e}")

    def pause(self):
        self.engine.stop()
        
    def stop(self):
        self.engine.stop()
        if os.path.exists(self.audio_file):
            os.remove(self.audio_file)
        
    def save(self):
        text = self.text_input.get("1.0", tk.END).strip()
        language = self.language_var.get()
        file_path = filedialog.asksaveasfilename(defaultextension=".mp3",
                                                 filetypes=[("MP3 files", "*.mp3"), ("WAV files", "*.wav")])
        if not text:
            messagebox.showwarning("Input Error", "Please enter text to convert to speech.")
            return
        
        if file_path:
            try:
                if language == "en":
                    self.engine.save_to_file(text, file_path)
                    self.engine.runAndWait()
                else:
                    tts = gTTS(text=text, lang=language, slow=False)
                    tts.save(file_path)
                    audio = AudioSegment.from_file(file_path, format="mp3")
                    audio.export(file_path, format=os.path.splitext(file_path)[1][1:])
                messagebox.showinfo("Success", "Audio file saved successfully.")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save audio file: {e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = TextToSpeechApp(root)
    root.mainloop()
